In [788]:
dict(zip('Serial Number	CountAbsence	DaysAbsenceAvg	Leave	MonthPerLoan	CountLoan	Year Counts	AllCounts	Last Duration	NumCer	LastDuration	AttendCount	MaxLevel	Other	EDU	BUSINESS	Finance	ENG_IND	Law	COM	Science	ENG_PET/CHEM	ENG_COM/ELEC	CHEM	ENG_CIVIL	Social	Languages	ENG_OTHER	ENG_CHEM	ENG_ENV	Inst	Hire type	<add>	Source	WorkDuration'.split('\t'),
'SerialNumber	Avg_TotalAbsensePerYear	Avg_NumDaysPerAbsense	Leave	Avg_MonthPerLoan	CountLoan	Avg_EduAllowance	TotalEduAllowance	NumYear_SinceLastEduAllance	NumCer	NumYear_SinceLastEdu	TotalEduAttend	MaxEduDegreeLevel	EduBranch_Other	EduBranch_EDU	EduBranch_BUSINESS	EduBranch_Finance	EduBranch_ENG_IND	EduBranch_Law	EduBranch_COM	EduBranch_Science	EduBranch_ENG_PET_CHEM	EduBranch_ENG_COM_ELEC	EduBranch_CHEM	EduBranch_ENG_CIVIL	EduBranch_Social	EduBranch_Languages	EduBranch_ENG_OTHER	EduBranch_ENG_CHEM	EduBranch_ENG_ENV	EduInstituteGroup	HireTypeGroup	ExceptionDescription	HireSourceGroup	WorkDuration'.split('\t')
        ))


{'<add>': 'ExceptionDescription',
 'AllCounts': 'TotalEduAllowance',
 'AttendCount': 'TotalEduAttend',
 'BUSINESS': 'EduBranch_BUSINESS',
 'CHEM': 'EduBranch_CHEM',
 'COM': 'EduBranch_COM',
 'CountAbsence': 'Avg_TotalAbsensePerYear',
 'CountLoan': 'CountLoan',
 'DaysAbsenceAvg': 'Avg_NumDaysPerAbsense',
 'EDU': 'EduBranch_EDU',
 'ENG_CHEM': 'EduBranch_ENG_CHEM',
 'ENG_CIVIL': 'EduBranch_ENG_CIVIL',
 'ENG_COM/ELEC': 'EduBranch_ENG_COM_ELEC',
 'ENG_ENV': 'EduBranch_ENG_ENV',
 'ENG_IND': 'EduBranch_ENG_IND',
 'ENG_OTHER': 'EduBranch_ENG_OTHER',
 'ENG_PET/CHEM': 'EduBranch_ENG_PET_CHEM',
 'Finance': 'EduBranch_Finance',
 'Hire type': 'HireTypeGroup',
 'Inst': 'EduInstituteGroup',
 'Languages': 'EduBranch_Languages',
 'Last Duration': 'NumYear_SinceLastEduAllance',
 'LastDuration': 'NumYear_SinceLastEdu',
 'Law': 'EduBranch_Law',
 'Leave': 'Leave',
 'MaxLevel': 'MaxEduDegreeLevel',
 'MonthPerLoan': 'Avg_MonthPerLoan',
 'NumCer': 'NumCer',
 'Other': 'EduBranch_Other',
 'Science': 'EduBranch_

# import

In [0]:
import datetime

import numpy as np
import pandas as pd
import os

import io

In [1]:
from google.colab import files

# uploaded = files.upload()
uploaded = files.upload()

In [12]:
! pip install xlrd

    100% |████████████████████████████████| 112kB 2.5MB/s 


In [0]:
DATA_FILE = pd.ExcelFile('Data-Thailand-HR_masked.xlsx')

# create target

In [0]:
WORKF = pd.read_excel(DATA_FILE, 'TH workforce', encoding='utf-8')

In [0]:
LEAVE_ACTIONS = [
    'LEAVE THE COMPANY',
    'Withdrawn Chg. (Migration)',
#     'RETIREMENT NOT PAID',
#     'INTERNAL MOVE',
#     'RETURN TO HOME COMPANY',
#     'CHANGE EMPLOYEE SITUATION',
#     'Hiring Active (Migration)',
#     'RE-ENTRY IN THE COMPANY',
#     'ENTRY IN THE COMPANY',
]

In [0]:
WORKF['Leave'] = WORKF['Personnel Action Text'].map({a:1 for a in LEAVE_ACTIONS}).fillna(0)

In [0]:
target = WORKF[['Serial Number', 'Leave']].set_index('Serial Number')

In [593]:
target.head()

,Leave
Serial Number,
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0


# Feature Extraction (Sheet: Absence)

## load data

In [0]:
features = dict()

In [0]:
GTYPE_FILE = pd.ExcelFile('group_type.xlsx')

GTYPE = dict()
GTYPE['AbsenseTypeGroup'] = pd.read_excel(GTYPE_FILE, 'Absence')

In [0]:
ABS = pd.read_excel(DATA_FILE, 'Absence').iloc[:, :5]

ABS.set_index('Attendance or Absence Type', inplace=True)
ABS['AbsenseTypeGroup'] = GTYPE['AbsenseTypeGroup'].set_index('Attendance or Absence Type').AbsenseTypeGroup
ABS.reset_index(inplace=True)
ABS.columns = ABS.columns.str.strip()

In [0]:
abs_types = {
    'Maternity':'Maternity',
    'Other':'Other',
    'Personal Leave':'Personal',
    'Sick/Death':'Vital',
    'Vacation':'Vacation',
}

In [0]:
ABS['DaysNew'] = (ABS['End date'] - ABS['Start date']).dt.days + 1

## overall

### count

In [0]:
WORKF['WorkDuration'] = WORKF['Personnel action - Year'] - WORKF['Tech Date of Entry'].dt.year
WORKF.loc[~WORKF.Leave.astype(bool), 'WorkDuration'] = 2018 - WORKF['Tech Date of Entry'].dt.year

In [0]:
features['CountAbsence'] = (ABS.groupby('Serial Number')['Attendance or Absence Type'].count()/WORKF.set_index('Serial Number').WorkDuration).fillna(0)

In [0]:
features['CountAbsence'].loc[(features['CountAbsence'] > 100000)] = np.nan

In [0]:
gcount = ABS.groupby(['AbsenseTypeGroup', 'Serial Number'])['Attendance or Absence Type'].count()


for k, v in abs_types.items():
    features['CountAbsence'+v] = gcount[k]

### days

In [0]:
features['DaysAbsence'] = ABS.groupby('Serial Number')['DaysNew'].sum()

In [0]:
gday = ABS.groupby(['AbsenseTypeGroup', 'Serial Number'])['DaysNew'].sum()

for k, v in abs_types.items():
    features['DaysAbsence'+v] = gday[k]

### avg days

In [0]:
features['DaysAbsenceAvg'] = ABS.groupby('Serial Number')['DaysNew'].mean()

gday = ABS.groupby(['AbsenseTypeGroup', 'Serial Number'])['DaysNew'].mean()

for k, v in abs_types.items():
    features['DaysAbsenceAvg'+v] = gday[k]

## rolling

### create last action date

In [0]:
WORKF['LastDate'] = pd.to_datetime(
    (WORKF['Personnel action - Year']+(WORKF['Personnel action - Month'] == 12))*10000+(WORKF['Personnel action - Month']+1).replace({13:1})*100+1,format='%Y%m%d') - datetime.timedelta(days=1)

In [0]:
last_act_dt = WORKF.set_index('Serial Number')['LastDate']

In [606]:
last_act_dt.head()

Serial Number
1   1998-09-30
2   1998-08-31
3   1998-02-28
4   2000-02-29
5   2000-05-31
Name: LastDate, dtype: datetime64[ns]

### create features

In [0]:
def create_rolling_absence(back):
  # bound
  _abs = ABS.set_index('Serial Number').assign(LastDate=last_act_dt).reset_index()
  _abs = _abs.loc[
      ((_abs.LastDate - datetime.timedelta(days=365*back)) <= _abs['Start date']) &\
      (_abs['Start date'] <= _abs.LastDate)
  ]

  back_str = str(back)
  
  
  # count
  features['CountAbsenceBack'+back_str] = _abs.groupby('Serial Number')['Attendance or Absence Type'].count()
  gcount = _abs.groupby(['AbsenseTypeGroup', 'Serial Number'])['Attendance or Absence Type'].count()

  for k, v in abs_types.items():
      features['CountAbsence'+v+'Back'+back_str] = gcount[k]

      
  # days
  features['DaysAbsenceBack'+back_str] = _abs.groupby('Serial Number')['DaysNew'].sum()
  gday = _abs.groupby(['AbsenseTypeGroup', 'Serial Number'])['DaysNew'].sum()

  for k, v in abs_types.items():
      features['DaysAbsence'+v+'Back'+back_str] = gday[k]
  
  
  # days avg
  features['DaysAbsenceAvgBack'+back_str] = _abs.groupby('Serial Number')['DaysNew'].mean()
  gday = _abs.groupby(['AbsenseTypeGroup', 'Serial Number'])['DaysNew'].mean()

  for k, v in abs_types.items():
      features['DaysAbsenceAvg'+v+'Back'+back_str] = gday[k]

In [0]:
for i in range(1, 4):
  create_rolling_features(i)

## merge data

In [0]:
Absence = list()

Absence.append(target.Leave.rename('Leave'))
for name, series in features.items():
  Absence.append(series.rename(name))

Absence = pd.concat(Absence, axis=1).fillna(0)


In [777]:
Absence.head()

,Leave,DaysAbsencePersonalBack1,DaysAbsenceVitalBack2,DaysAbsencePersonalBack3,DaysAbsenceVitalBack1,DaysAbsenceOtherBack2,DaysAbsenceOtherBack3,DaysAbsenceOtherBack1,DaysAbsenceBack1,DaysAbsenceMaternityBack2,...,CountAbsenceMaternityBack1,CountAbsenceMaternityBack2,CountAbsenceMaternityBack3,DaysAbsenceVacationBack2,CountAbsenceBack3,DaysAbsenceVacationBack1,DaysAbsenceAvgVital,CountAbsenceBack1,CountAbsenceBack2,DaysAbsence
Serial Number,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Feature Extraction (Sheet: Loan)

In [0]:
LOAN = pd.read_excel(DATA_FILE, 'Housing Loan')

LOAN['EndDateShift'] = LOAN.sort_values('End date').groupby('Serial Number')['End date'].apply(lambda t: t.shift(1))
LOAN['gap'] = (LOAN['End date'] - LOAN['EndDateShift']).dt.days

LOAN['Start'] = LOAN.gap.isnull() | (LOAN.gap >= 60)

In [612]:
LOAN[['Serial Number', 'End date', 'gap', 'Start']].head()

,Serial Number,End date,gap,Start
0,34,2002-10-31,NaN,True
1,34,2002-11-29,29.0,False
2,34,2002-12-30,31.0,False
3,34,2003-01-31,32.0,False
4,34,2003-02-28,28.0,False


In [613]:
Loan = pd.concat([
    (LOAN.groupby('Serial Number').Start.count()/LOAN.groupby('Serial Number').Start.sum()).rename('MonthPerLoan'),
    LOAN.groupby('Serial Number').Start.sum().rename('CountLoan'),
], axis=1)
Loan.head()

,MonthPerLoan,CountLoan
Serial Number,,
34,12.0,1.0
105,13.0,1.0
134,52.0,1.0
136,23.0,1.0
156,18.0,1.0


# Feature Extraction (Sheet: HireType)

In [0]:
HIRETYPE = pd.read_excel(GTYPE_FILE, 'HireType')

In [0]:
HIRE = pd.read_excel(DATA_FILE, 'Hire Type & Sources').iloc[:, :5]

HIRE.set_index(['Statistics Indicator Description (0033)','Exception Description (0033)'], inplace=True)
HIRE['ExceptionTypeGroup'] = HIRETYPE.set_index(['Statistics Indicator Description (0033)','Exception Description (0033)']).ExceptionTypeGroup
HIRE.reset_index(inplace=True)
HIRE.columns = HIRE.columns.str.strip()

In [0]:
HIRE= HIRE[['Serial Number','Statistics Indicator Description (0033)','ExceptionTypeGroup']]

In [619]:
HIRE.head()

,Serial Number,Statistics Indicator Description (0033),ExceptionTypeGroup
0,72,Hire type,Inexperienced Hire
1,134,Hire type,Inexperienced Hire
2,479,Source,Other
3,479,Hire type,Experienced Hire
4,615,Source,Other


In [0]:
hire_df = HIRE.pivot_table(values='ExceptionTypeGroup', index=HIRE['Serial Number'], columns='Statistics Indicator Description (0033)', aggfunc='first').sort_index()

In [0]:
del hire_df.columns.name

In [790]:
hire_df.head()

,Hire type,Source
Serial Number,,
72,Inexperienced Hire,None
134,Inexperienced Hire,None
479,Experienced Hire,Other
615,Experienced Hire,Other
694,Experienced Hire,Other


# Feature Extraction (Sheet: Edu - Cert and Branch Type)

In [0]:
EDU = pd.read_excel(DATA_FILE, 'Education').iloc[:, :6]

In [0]:
CertType_df = pd.read_excel(GTYPE_FILE, 'CertType')
BranchType_df = pd.read_excel(GTYPE_FILE, 'BranchType')

In [630]:
EDU.set_index('Certificate Text', inplace=True)
EDU['CertTypeGroup'] = CertType_df.set_index('Certificate Text')['Certificate Text Type Group']
EDU['CertLevel'] = CertType_df.set_index('Certificate Text')['Level']
EDU.reset_index(inplace=True)

EDU.set_index('Branch of study 1 Text', inplace=True)
EDU['BranchTypeGroup'] = BranchType_df.set_index('Branch of study 1 Text')['Branch Type Group']
EDU.reset_index(inplace=True)

EDU.columns = EDU.columns.str.strip()
EDU.head()

,Branch of study 1 Text,Certificate Text,Serial Number,Start Date (0022),End Date (0022),Institute/location,CertTypeGroup,CertLevel,BranchTypeGroup
0,General Studies,Attended,1,1950-01-01,1953-01-01 00:00:00,SUANKWONG SCH.,Attend,NaN,Other
1,No Specific Branch,Attended,2,1953-01-01,1957-01-01 00:00:00,VIENGPROU VITHAYA SCH.,Attend,NaN,Other
2,No Specific Branch,Attended,3,1949-01-01,1952-01-01 00:00:00,WADTEPSIRIN SCH.,Attend,NaN,Other
3,Education,Attended,4,1956-01-01,1959-01-01 00:00:00,WATTANA VITHAYA ACADEMY.,Attend,NaN,EDU
4,Business-Economics,Bachelor,5,1956-06-01,1960-03-31 00:00:00,THAMMASAT UNIVERSITY,Bachelor,3.0,BUSINESS


In [0]:
attendLv_df = EDU[EDU['CertLevel'].isnull()] 
otherLv_df = EDU[EDU['CertLevel'].notnull()]

In [632]:
attendCountLv_df = attendLv_df.groupby('Serial Number').size().reset_index(name='AttendCount')
attendCountLv_df.head()

,Serial Number,AttendCount
0,1,1
1,2,1
2,3,1
3,4,1
4,6,1


In [633]:
otherMaxLv_df = otherLv_df.groupby('Serial Number')['CertLevel'].max().reset_index(name='MaxLevel')
otherMaxLv_df.head()

,Serial Number,MaxLevel
0,5,3.0
1,7,2.0
2,11,3.0
3,12,2.0
4,16,3.0


In [634]:
cert_df = pd.merge(attendCountLv_df, otherMaxLv_df,  how='outer', left_on=['Serial Number'], right_on = ['Serial Number']).fillna(0).sort_values('Serial Number')
cert_df.head()

,Serial Number,AttendCount,MaxLevel
0,1,1.0,0.0
1,2,1.0,0.0
2,3,1.0,0.0
3,4,1.0,0.0
732,5,0.0,3.0


In [635]:
branchTypeGroup_df = pd.get_dummies(EDU['BranchTypeGroup'])
branchType_df = pd.concat([EDU, branchTypeGroup_df], axis=1)
branchType_df.head()

,Branch of study 1 Text,Certificate Text,Serial Number,Start Date (0022),End Date (0022),Institute/location,CertTypeGroup,CertLevel,BranchTypeGroup,BUSINESS,...,ENG_ENV,ENG_IND,ENG_OTHER,ENG_PET/CHEM,Finance,Languages,Law,Other,Science,Social
0,General Studies,Attended,1,1950-01-01,1953-01-01 00:00:00,SUANKWONG SCH.,Attend,NaN,Other,0,...,0,0,0,0,0,0,0,1,0,0
1,No Specific Branch,Attended,2,1953-01-01,1957-01-01 00:00:00,VIENGPROU VITHAYA SCH.,Attend,NaN,Other,0,...,0,0,0,0,0,0,0,1,0,0
2,No Specific Branch,Attended,3,1949-01-01,1952-01-01 00:00:00,WADTEPSIRIN SCH.,Attend,NaN,Other,0,...,0,0,0,0,0,0,0,1,0,0
3,Education,Attended,4,1956-01-01,1959-01-01 00:00:00,WATTANA VITHAYA ACADEMY.,Attend,NaN,EDU,0,...,0,0,0,0,0,0,0,0,0,0
4,Business-Economics,Bachelor,5,1956-06-01,1960-03-31 00:00:00,THAMMASAT UNIVERSITY,Bachelor,3.0,BUSINESS,1,...,0,0,0,0,0,0,0,0,0,0


In [636]:
branchType_list = EDU['BranchTypeGroup'].unique()
branchType_list

array([u'Other', u'EDU', u'BUSINESS', u'Finance', u'ENG_IND', u'Law',
       u'COM', u'Science', u'ENG_PET/CHEM', u'ENG_COM/ELEC', u'CHEM',
       u'ENG_CIVIL', u'Social', u'Languages', u'ENG_OTHER', u'ENG_CHEM',
       u'ENG_ENV'], dtype=object)

In [637]:
branchType_df = branchType_df.groupby(['Serial Number'])[branchType_list].max().reset_index()
branchType_df[branchType_df.index == 4839]

,Serial Number,Other,EDU,BUSINESS,Finance,ENG_IND,Law,COM,Science,ENG_PET/CHEM,ENG_COM/ELEC,CHEM,ENG_CIVIL,Social,Languages,ENG_OTHER,ENG_CHEM,ENG_ENV
4839,4883,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [695]:
cert_branch_df = pd.merge(cert_df, branchType_df,  how='outer', left_on=['Serial Number'], right_on = ['Serial Number'])
cert_branch_df.head()

,Serial Number,AttendCount,MaxLevel,Other,EDU,BUSINESS,Finance,ENG_IND,Law,COM,Science,ENG_PET/CHEM,ENG_COM/ELEC,CHEM,ENG_CIVIL,Social,Languages,ENG_OTHER,ENG_CHEM,ENG_ENV
0,1,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,1.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0.0,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [696]:
cert_branch_df[cert_branch_df['Serial Number'] == 4839]

,Serial Number,AttendCount,MaxLevel,Other,EDU,BUSINESS,Finance,ENG_IND,Law,COM,Science,ENG_PET/CHEM,ENG_COM/ELEC,CHEM,ENG_CIVIL,Social,Languages,ENG_OTHER,ENG_CHEM,ENG_ENV
4799,4839,3.0,3.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [697]:
cert_branch_df.head()

,Serial Number,AttendCount,MaxLevel,Other,EDU,BUSINESS,Finance,ENG_IND,Law,COM,Science,ENG_PET/CHEM,ENG_COM/ELEC,CHEM,ENG_CIVIL,Social,Languages,ENG_OTHER,ENG_CHEM,ENG_ENV
0,1,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,1.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,0.0,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
cert_branch_df.set_index('Serial Number', inplace=True)

# Feature Extraction (Sheet: Edu - Inst)

In [0]:
inst = EDU['Institute/location']

In [0]:
edu_patterns = dict(
    UNIV = r'UNIV\.|\WU\.|^U\.|UNIV|ERSITY|UNI$',
    SCHL = r'SCH\S|\WSCH$|SUKSA',
    INST = r'\WINST|^INST',
    COLL = r'\WCOLL|^COLL|COL$|COLLEGE',
    CAMP = r'C..PUS',
    ACAD = r'ACAD',
    NIDA = r'NIDA',
    SASI = r'SASIN',
)

In [0]:
for name, patt in edu_patterns.items():
  EDU.loc[EDU['Institute/location'].str.contains(patt), 'Inst'] = name
EDU['Inst'].fillna('OTHR', inplace=True)

In [0]:
inst_highest = EDU.groupby('Serial Number').apply(lambda t: t[t.CertLevel == t.CertLevel.max()]).reset_index(drop=True)

In [0]:
Inst = inst_highest.sort_values('Start Date (0022)', ascending=False).drop_duplicates(subset=['Serial Number']).set_index('Serial Number')['Inst'].to_frame()

#  Feature Extraction (Sheet: Edu - Cert and Branch Type II)



In [0]:
EDU = EDU[['Serial Number','Start Date (0022)','End Date (0022)','Institute/location','Certificate Text','Branch of study 1 Text']]

In [648]:
now = datetime.datetime.today().year
now

2018

In [0]:
EDU['End Date (0022)'] = EDU['End Date (0022)'].astype('str') 

In [650]:
EDU.head()

,Serial Number,Start Date (0022),End Date (0022),Institute/location,Certificate Text,Branch of study 1 Text
0,1,1950-01-01,1953-01-01 00:00:00,SUANKWONG SCH.,Attended,General Studies
1,2,1953-01-01,1957-01-01 00:00:00,VIENGPROU VITHAYA SCH.,Attended,No Specific Branch
2,3,1949-01-01,1952-01-01 00:00:00,WADTEPSIRIN SCH.,Attended,No Specific Branch
3,4,1956-01-01,1959-01-01 00:00:00,WATTANA VITHAYA ACADEMY.,Attended,Education
4,5,1956-06-01,1960-03-31 00:00:00,THAMMASAT UNIVERSITY,Bachelor,Business-Economics


In [0]:
EDU['Start date Y'] = pd.DatetimeIndex(EDU['Start Date (0022)']).year
EDU['End date Y'] = EDU['End Date (0022)'].str[:4].astype(int)

In [652]:
EDU['End date Y'].unique()

array([1953, 1957, 1952, 1959, 1960, 1961, 1948, 1962, 1990, 1963, 1964,
       1989, 1965, 1991, 1968, 1966, 1975, 1954, 1967, 1955, 1950, 1970,
       1958, 1986, 1969, 1993, 1956, 1988, 1971, 1976, 1972, 1979, 1973,
       1974, 1992, 1984, 1994, 1977, 1978, 1985, 1980, 1987, 1981, 1982,
       1983, 2000, 1998, 1995, 1999, 2006, 2005, 1996, 1997, 2009, 2013,
       2007, 2001, 2014, 2008, 2010, 2004, 2003, 2002, 2011, 2012, 2200,
       2017, 2015, 2016, 9999])

In [0]:
EDU = EDU[ ( EDU['End date Y'] <= now )  ]

In [654]:
EDU['End date Y'].unique()

array([1953, 1957, 1952, 1959, 1960, 1961, 1948, 1962, 1990, 1963, 1964,
       1989, 1965, 1991, 1968, 1966, 1975, 1954, 1967, 1955, 1950, 1970,
       1958, 1986, 1969, 1993, 1956, 1988, 1971, 1976, 1972, 1979, 1973,
       1974, 1992, 1984, 1994, 1977, 1978, 1985, 1980, 1987, 1981, 1982,
       1983, 2000, 1998, 1995, 1999, 2006, 2005, 1996, 1997, 2009, 2013,
       2007, 2001, 2014, 2008, 2010, 2004, 2003, 2002, 2011, 2012, 2017,
       2015, 2016])

In [0]:
EDU['Use time'] = EDU['End date Y'].astype('int64') - EDU['Start date Y'].astype('int64')

In [0]:
x = EDU.groupby(['Serial Number'])['End date Y'].max().reset_index(name='LatestYear')

In [657]:
x.head(10)

,Serial Number,LatestYear
0,1,1953
1,2,1957
2,3,1952
3,4,1959
4,5,1960
5,6,1953
6,7,1961
7,8,1961
8,9,1948
9,10,1962


In [0]:
y = EDU.groupby(['Serial Number'])['Certificate Text'].count().reset_index(name='NumCer')

In [0]:
EDUs = pd.merge(x, y,  how='outer', left_on=['Serial Number'], right_on = ['Serial Number'])

In [0]:
LEAVE_ACTIONS = [
    'LEAVE THE COMPANY',
    'Withdrawn Chg. (Migration)',
]
WORKF['Leave'] = WORKF['Personnel Action Text'].map({a:1 for a in LEAVE_ACTIONS}).fillna(0)
target = WORKF[['Serial Number', 'Leave', 'Personnel action - Year']].set_index('Serial Number')

df = target.join(EDUs.reset_index().set_index('Serial Number'))
leave_df = df[df['Leave'] == 1.0]
not_leave_df = df[df['Leave'] != 1.0]

In [661]:
leave_df['LastDuration'] = leave_df['Personnel action - Year'] - leave_df['LatestYear']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [662]:
not_leave_df['LastDuration'] = now - not_leave_df['LatestYear']
not_leave_df.head()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Leave,Personnel action - Year,index,LatestYear,NumCer,LastDuration
Serial Number,,,,,,
40,0.0,2007,39.0,1966.0,1.0,52.0
72,0.0,2009,71.0,1964.0,1.0,54.0
121,0.0,2009,117.0,1967.0,1.0,51.0
134,0.0,2009,129.0,1965.0,1.0,53.0
136,0.0,2010,131.0,1962.0,1.0,56.0


In [666]:
pd.concat([leave_df,not_leave_df]).head()

,Leave,Personnel action - Year,index,LatestYear,NumCer,LastDuration
Serial Number,,,,,,
1,1.0,1998,0.0,1953.0,1.0,45.0
2,1.0,1998,1.0,1957.0,1.0,41.0
3,1.0,1998,2.0,1952.0,1.0,46.0
4,1.0,2000,3.0,1959.0,1.0,41.0
5,1.0,2000,4.0,1960.0,1.0,40.0


In [0]:
edu_time_df = pd.concat([leave_df,not_leave_df])[['NumCer', 'LastDuration']]

In [668]:
edu_time_df.head()

,NumCer,LastDuration
Serial Number,,
1,1.0,45.0
2,1.0,41.0
3,1.0,46.0
4,1.0,41.0
5,1.0,40.0


#  Feature Extraction (Sheet: Edu Allowance)

In [0]:
EDUAW = pd.read_excel(DATA_FILE, 'Education allowance').iloc[:, :4]

In [28]:
EDUAW.head()

,Serial Number,Start date,End date,Wage Type
0,446,2004-09-30,2004-09-30,Edu Assist'nce-Public
1,446,2006-06-26,2006-06-26,Edu Assist'nce-Public
2,446,2006-06-30,2006-06-30,Edu Assist'nce-Public
3,500,2005-04-30,2005-04-30,Edu Assist'nce-Public
4,500,2005-11-30,2005-11-30,Edu Assist'nce-Public


In [0]:
rename_col = {
    'Serial Number':'SerialNumber',
}

EDUAW.rename(rename_col, axis=1, inplace=True)

In [31]:
now = datetime.datetime.today().year
now

2018

In [0]:
EDUAW['Start date Y'] = EDUAW['Start date'].dt.year
EDUAW['End date Y'] = EDUAW['End date'].dt.year
EDUAW['Present'] = now

EDUAW['NumYear_SinceLastEduAllowance'] = EDUAW['Present'] - EDUAW['End date Y']

In [0]:
eduaw_features = pd.DataFrame()

In [0]:
eduaw_features['LastStartYear'] = EDUAW.groupby(['SerialNumber'])['Start date Y'].max()
eduaw_features['TotalEduAllowance'] = EDUAW.groupby(['SerialNumber']).size()

In [0]:
z = EDUAW.groupby(['SerialNumber','Start date Y']).size().reset_index(name='Year Counts')

In [677]:
EDUAW_df = x.set_index('Serial Number').join(y.set_index(['Serial Number']))
EDUAW_df.reset_index(inplace=True)
EDUAW_df.set_index(['Serial Number','MaxYear'], inplace=True)
EDUAW_df.head(5)

,,All Counts
Serial Number,MaxYear,
446,2006,3
500,2007,3
540,2013,1
543,2013,1
546,2014,1


In [0]:
z = z.rename({'Start date Y':'MaxYear'}, axis=1).set_index(['Serial Number','MaxYear'])

In [0]:
EDUAW_df = z.assign(AllCounts=EDUAW_df['All Counts']).dropna()

In [0]:
LEAVE_ACTIONS = [
    'LEAVE THE COMPANY',
    'Withdrawn Chg. (Migration)',
#     'RETIREMENT NOT PAID',
#     'INTERNAL MOVE',
#     'RETURN TO HOME COMPANY',
#     'CHANGE EMPLOYEE SITUATION',
#     'Hiring Active (Migration)',
#     'RE-ENTRY IN THE COMPANY',
#     'ENTRY IN THE COMPANY',
]
WORKF['Leave'] = WORKF['Personnel Action Text'].map({a:1 for a in LEAVE_ACTIONS}).fillna(0)
target = WORKF[['Serial Number', 'Leave', 'Personnel action - Year']].set_index('Serial Number')

df = target.join(EDUAW_df.reset_index().set_index('Serial Number'))
leave_df = df[df['Leave'] == 1.0].dropna()
not_leave_df = df[df['Leave'] != 1.0]

In [0]:
leave_df['Last Duration'] = leave_df['Personnel action - Year'] - leave_df['MaxYear']
now = datetime.datetime.today().year

In [682]:
now = datetime.datetime.today().year
now

not_leave_df['Last Duration'] = now - not_leave_df['MaxYear']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
edu_allow_df = pd.concat([leave_df,not_leave_df])[['Year Counts', 'AllCounts', 'Last Duration']]

In [693]:

edu_allow_df.head()

,Year Counts,AllCounts,Last Duration
Serial Number,,,
659,1.0,1.0,6.0
962,1.0,2.0,0.0
963,1.0,2.0,4.0
1035,1.0,2.0,3.0
1234,2.0,3.0,4.0


# Merge All

In [685]:
Absence.head()

,Leave,DaysAbsencePersonalBack1,DaysAbsenceVitalBack2,DaysAbsencePersonalBack3,DaysAbsenceVitalBack1,DaysAbsenceOtherBack2,DaysAbsenceOtherBack3,DaysAbsenceOtherBack1,DaysAbsenceBack1,DaysAbsenceMaternityBack2,...,CountAbsenceMaternityBack1,CountAbsenceMaternityBack2,CountAbsenceMaternityBack3,DaysAbsenceVacationBack2,CountAbsenceBack3,DaysAbsenceVacationBack1,DaysAbsenceAvgVital,CountAbsenceBack1,CountAbsenceBack2,DaysAbsence
Serial Number,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [785]:
Loan.head()

,MonthPerLoan,CountLoan
Serial Number,,
34,12.0,1.0
105,13.0,1.0
134,52.0,1.0
136,23.0,1.0
156,18.0,1.0


In [687]:
hire_df.head()

,Hire type,Source
Serial Number,,
72,Inexperienced Hire,None
134,Inexperienced Hire,None
479,Experienced Hire,Other
615,Experienced Hire,Other
694,Experienced Hire,Other


In [688]:
Inst.head()

,Inst
Serial Number,
5028,UNIV
4993,UNIV
4338,UNIV
4522,UNIV
4969,UNIV


In [689]:
cert_branch_df.head()

,AttendCount,MaxLevel,Other,EDU,BUSINESS,Finance,ENG_IND,Law,COM,Science,ENG_PET/CHEM,ENG_COM/ELEC,CHEM,ENG_CIVIL,Social,Languages,ENG_OTHER,ENG_CHEM,ENG_ENV
Serial Number,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0.0,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [690]:
edu_time_df.head()

,NumCer,LastDuration
Serial Number,,
1,1.0,45.0
2,1.0,41.0
3,1.0,46.0
4,1.0,41.0
5,1.0,40.0


In [694]:
edu_allow_df.head()
edu_time_df.head()
cert_branch_df.head()
Inst.head()
hire_df

,Year Counts,AllCounts,Last Duration
Serial Number,,,
659,1.0,1.0,6.0
962,1.0,2.0,0.0
963,1.0,2.0,4.0
1035,1.0,2.0,3.0
1234,2.0,3.0,4.0


In [0]:
MergeAll = pd.concat([
    Absence[['CountAbsence', 'DaysAbsenceAvg', 'Leave']],
    Loan,
    edu_allow_df,
    edu_time_df,
    cert_branch_df,
    Inst,
    hire_df,
    WORKF.set_index('Serial Number')['WorkDuration'],
    WORKF.set_index('Serial Number')['Personnel action - Year'],
], axis=1)

In [779]:
MergeAll.shape

(5038, 34)

In [0]:
MergeAll = MergeAll[MergeAll['Personnel action - Year'] >= 2000].drop('Personnel action - Year', axis=1)

--------------

In [0]:
MergeAll.to_csv('MergeAll.csv')

In [0]:
files.download('MergeAll.csv')

In [708]:
MergeAll.columns[MergeAll.columns.str.contains('person', case=False)]

Index([u'DaysAbsencePersonalBack1', u'DaysAbsencePersonalBack3',
       u'DaysAbsencePersonalBack2', u'CountAbsencePersonal',
       u'DaysAbsenceAvgPersonal', u'CountAbsencePersonalBack1',
       u'CountAbsencePersonalBack2', u'CountAbsencePersonalBack3',
       u'DaysAbsencePersonal'],
      dtype='object')